In [3]:
# -*- coding: utf-8 -*-
"""
Created on Thu Apr 30 21:46:05 2020

@author: Rocky
"""

import pandas as pd 
from sklearn.model_selection import cross_val_score
#from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt 

train_data_source = pd.read_csv('C:\\Users\\Rocky\\Desktop\\train_data.csv')
train_Y = train_data_source['poi']
train_data_source.describe()

#觀察other欄位與其他欄位關係
corr_res_other  = train_data_source.corr()['other']
plt.scatter(train_data_source['total_payments'],train_data_source['other'])
corr_res_expenses  = train_data_source.corr()['expenses']
plt.scatter(train_data_source['expenses'],train_data_source['director_fees'])
corr_res_salary  = train_data_source.corr()['salary']
from sklearn.linear_model import LinearRegression
import numpy as np
plt.scatter(train_data_source['salary'],train_data_source['bonus'])
plt.scatter(train_data_source['salary'],train_data_source['exercised_stock_options'])
regression_train  = train_data_source[(~(train_data_source['salary'].isna() ) & (~train_data_source['exercised_stock_options'].isna() ) ) ]
regression_x = np.asarray(regression_train[['exercised_stock_options']])
regression_y =  np.asarray(regression_train[['salary']])
lr = LinearRegression()
lr.fit(regression_x,regression_y)
predict_data  = train_data_source[(train_data_source['salary'].isna() ) ]
pre_salary = lr.predict(np.asarray(predict_data[['exercised_stock_options']].fillna(0) ))
train_data_source.loc[train_data_source['salary'].isna(),'salary' ] = pre_salary




train_data_source['to_poi_ratio'] = train_data_source['from_poi_to_this_person'] / train_data_source['to_messages']
train_data_source['from_poi_ratio'] = train_data_source['from_this_person_to_poi'] / train_data_source['from_messages']
train_data_source['shared_poi_ratio'] = train_data_source['shared_receipt_with_poi'] / train_data_source['to_messages']
train_data_source['bonus_to_salary'] = train_data_source['bonus'] / train_data_source['salary']
train_data_source['bonus_to_total'] = train_data_source['bonus'] / train_data_source['total_payments']


non_fea = ['name','email_address','poi']
fea_col = list(train_data_source.columns)
fea_col.remove('name')
fea_col.remove('email_address')
fea_col.remove('poi')
# poi none data
fea_col.remove('loan_advances')
fea_col.remove('director_fees')
fea_col.remove('restricted_stock_deferred')
train_X = train_data_source[fea_col]

#salary_outlier=600000
#train_X.loc[train_X['salary'] >salary_outlier,'salary'] = train_X['salary'].median()
#train_X['salary'].fillna(train_X['salary'].median(),inplace=True)
#bonus_outlier=500000
#train_X.loc[train_X['bonus'] >salary_outlier,'bonus'] = train_X['bonus'].median()
#train_X['bonus'].fillna(train_X['bonus'].median(),inplace=True)
#long_term_incentive_outlier = 3000000
#train_X.loc[train_X['long_term_incentive'] >long_term_incentive_outlier,'long_term_incentive'] = train_X['long_term_incentive'].median()
#train_X['long_term_incentive'].fillna(train_X['long_term_incentive'].median(),inplace=True)

train_X.fillna(0,inplace=True)
estimator = RandomForestClassifier(criterion='entropy',max_features='auto')
score 






C:\Users\Rocky\Anaconda3\lib\site-packages\pandas\core\frame.py:4153: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


0.8936758893280633

In [2]:
#print(score)
#
#param_grid ={'criterion':['gini','entropy'],
#             'min_samples_leaf':[1,2,3],
#             'max_features' : ['auto','sqrt','log2']}

    
estimator.fit(train_X,train_Y)
test_data_source = pd.read_csv('C:\\Users\\Rocky\\Desktop\\test_features.csv')
#test_data_source['salary'].fillna(test_data_source['salary'].median(),inplace=True)
#test_data_source['bonus'].fillna(test_data_source['bonus'].median(),inplace=True)
#test_data_source['long_term_incentive'].fillna(test_data_source['long_term_incentive'].median(),inplace=True)
#test_data_source.fillna(test_data_source.median(),inplace=True)

salary_predict_data  = test_data_source[(test_data_source['salary'].isna() )]
test_pre_salary = lr.predict(np.asarray(salary_predict_data[['exercised_stock_options']].fillna(0) ))
test_data_source.loc[test_data_source['salary'].isna(),'salary' ] = test_pre_salary



test_data_source['to_poi_ratio'] = test_data_source['from_poi_to_this_person'] / test_data_source['to_messages']
test_data_source['from_poi_ratio'] = test_data_source['from_this_person_to_poi'] / test_data_source['from_messages']
test_data_source['shared_poi_ratio'] = test_data_source['shared_receipt_with_poi'] / test_data_source['to_messages']
test_data_source['bonus_to_salary'] = test_data_source['bonus'] / test_data_source['salary']
test_data_source['bonus_to_total'] = test_data_source['bonus'] / test_data_source['total_payments']

test_data_source.fillna(0,inplace=True)

test_data = test_data_source[fea_col]

predict_res = estimator.predict_proba(test_data)
native_res = predict_res[:,1]
res_df =  pd.DataFrame()
res_df['name'] = test_data_source['name']
res_df['poi'] = native_res
res_df.to_csv('using_lr_padding_value_v2.csv',index=False)

